In [1]:
from improved_find_data_start_row import *
from financials_clean_year_assign import * 
import expertForecastFunctionsCopy1 as ff




In [2]:
financial_data=improved_load_excel_with_autodetect("/Users/myself/Desktop/Walmart USA Serching for Growth/walmartHistoricalFinancials.xlsx")

financial_data=clean_cashflow_data(financial_data, 3)
financial_data=update_column_names(financial_data, base_year=2021,index_name = 'Financial Metric')



In [3]:
categories, base_mapping = ff.categorize_and_map_metrics(financial_data)
# assumptions = {
#     'Sales': {'type': 'YOY', 'rates': [0.0814, 0.0392, 0.0261, 0.0244, 0.0237, 0.0238]},
#     'Costs of goods sold': {'type': '% of revenue', 'rates': [0.7541, 0.7513, 0.7513, 0.7513, 0.7513, 0.7513]},
#     'Gross profit': {'type': 'direct', 'rates': None},  # Calculated directly as difference
#     'Operating and SG&A costs': {'type': '% of sales', 'rates': [0.1863, 0.1834, 0.1834, 0.1834, 0.1834, 0.1834]}}


assumptions = {
    'Operating revenue (Turnover)': {'type': 'YOY', 'rates': [0.015,0.015]},
    'Gross Profit': {'type': 'YOY', 'rates': [0.014,0.014]},
    'EBITDA': {'type': 'YOY', 'rates': [-0.022,-0.022]},
    'Cash flow [Net Income before D&A]]': {'type': 'YOY', 'rates': [-0.046,-0.046]},
    'Operating profit (loss) [EBIT]': {'type': 'YOY', 'rates': [-0.02,-0.02]},
    'Profit (loss) for the period [Net income]': {'type': 'YOY', 'rates': [-0.019,-0.019]}
}


# Base year for forecast
base_year = '2021'
forecast_years = ['2022F','2023F']


yoy_forecast_df = ff.forecast_financials(financial_data, assumptions, base_year, forecast_years)

financial_data = ff.handle_missing_values(financial_data)
#yoy_forecast_df, financial_data

In [4]:
historical_regression_forecast = ff.linear_regression_forecast(financial_data, forecast_years)

In [5]:
future_bases = {base: historical_regression_forecast.loc[base] for base in set(base_mapping.values())}
historical_ratios = {metric: financial_data.loc[metric] / financial_data.loc[base] for metric, base in base_mapping.items()}
historical_ratios = pd.DataFrame.from_dict(historical_ratios).dropna(axis='columns')

forecasts, projected_ratios = ff.calculate_forecasts(historical_ratios, forecast_years, future_bases, base_mapping)

In [6]:
forecasts

{'Non-current assets': array([132330.83611792, 130480.42789078]),
 'Intangible assets': array([  887.65253816, -2026.42820501]),
 'Tangible fixed assets': array([117953.37357136, 116853.10166323]),
 'Other non-current assets': array([13489.8100084 , 15653.75443255]),
 'Current assets': array([129104.64821257, 139466.08574374]),
 'Other current assets': array([7589.91884043, 7239.08623128]),
 'Total assets': array([261435.48433048, 269946.51363452]),
 'Net current assets': array([48455.40474382, 57351.17554869]),
 'Shareholders funds': array([102437.39025941, 104633.04622892]),
 'Other shareholders funds': array([100358.71458515, 102357.3090329 ]),
 'Non-current liabilities': array([78348.85060232, 83198.55721055]),
 'Other non-current liabilities': array([10334.09115626, 11454.09349175]),
 'Current liabilities': array([80649.24346875, 82114.91019505]),
 'Other current liabilities': array([27820.12045218, 28135.47812838]),
 "Total shareholders' funds and liabilities": array([261435.4843

In [10]:
result_forecast, result_percentages = ff.ratio_forecast_regression(financial_data, historical_regression_forecast, base_mapping, forecast_years)

result_forecast, result_percentages

(       Non-current assets  Intangible assets  Tangible fixed assets  \
 2022F       132330.836118         887.652538          117953.373571   
 2023F       130480.427891       -2026.428205          116853.101663   
 
        Other non-current assets  Current assets  Other current assets  \
 2022F              13489.810008   129104.648213           7589.918840   
 2023F              15653.754433   139466.085744           7239.086231   
 
         Total assets  Net current assets  Shareholders funds  \
 2022F  261435.484330        48455.404744       102437.390259   
 2023F  269946.513635        57351.175549       104633.046229   
 
        Other shareholders funds  ...  Operating revenue (Turnover)  \
 2022F             100358.714585  ...                 608590.506476   
 2023F             102357.309033  ...                 628506.432893   
 
                Sales   Gross profit  Operating profit (loss) [EBIT]  \
 2022F  606581.823362  130622.197558                    31928.701946   
 2

Index(['Non-current assets', 'Intangible assets', 'Tangible fixed assets',
       'Other non-current assets', 'Current assets', 'Stock', 'Debtors',
       'Other current assets', 'Of which cash and cash equivalent',
       'Total assets', 'Shareholders funds', 'Capital',
       'Other shareholders funds', 'Non-current liabilities', 'Long term debt',
       'Other non-current liabilities', 'Current liabilities',
       'Loans & short-term debt', 'Creditors', 'Other current liabilities',
       'Total shareholders' funds and liabilities', 'Working capital',
       'Net current assets', 'Enterprise value', 'Number of employees',
       'Operating revenue (Turnover)', 'Sales', 'Costs of goods sold',
       'Gross profit', 'Other operating expense (income)',
       'Operating profit (loss) [EBIT]', 'Financial profit (loss)',
       'Financial revenue', 'Financial expenses',
       'Profit (loss) before tax [PBT]', 'Income tax expenses (benefit)',
       'Profit (loss) after tax [PAT]',
    

In [ ]:
int(''.join(filter(str.isdigit, '2022F'))) - int(base_year)

In [ ]:
def five_year_forecast_financials(df, assumptions, base_year, forecast_years):
    """
    Generates financial forecasts based on provided assumptions and historical data.

    Parameters:
        df (DataFrame): Historical financial data.
        assumptions (dict): Growth assumptions for various financial metrics.
        base_year (str): The base year for the forecast (last historical year).
        forecast_years (list): List of years for which the forecast is to be made.
        
    Returns:
        DataFrame: Forecasted financials including CAGR for the forecast period.
    """
    
    forecast_df = pd.DataFrame(index=df.index, columns=forecast_years)
    last_values = df[base_year].copy()

    for year in forecast_years:
        for metric in last_values.index
            if metric in assumptions.items():
                for values in assumptions.items(metric):
                    base_value = last_values.get(metric, None)
                        if base_value is not None:
                            growth_rate = values['rates'][int(''.join(filter(str.isdigit, year))) - int(base_year) - 1]
                            forecast_value = base_value * (1 + growth_rate)
                            forecast_df.loc[metric, year] = forecast_value
                            last_values[metric] = forecast_value
            else: 
                growth_rate = 
    calculate_cagr(df, forecast_df, base_year, forecast_years)  # Calculate CAGR and add as a column

    return forecast_df


def calculate_historical_cagr(df, base_year, period):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with forecast data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year-period, base_year
    cagr_df = pd.DataFrame(index=df.index, columns=np.arange(s,b,1)
    start_year, end_year = base_year-period, base_year
    num_years = period
    df['CAGR'] = ((df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100


def calculate_cagr(df, forecast_df, base_year, forecast_years, ):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with forecast data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year, forecast_years[-1]
    num_years = len(forecast_years)
    forecast_df['CAGR'] = ((forecast_df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100


In [ ]:
import pandas as pd

def forecast_financials(df, assumptions, base_year, forecast_years, cagr_period=5):
    """
    Generates financial forecasts based on provided assumptions, historical data,
    and rolling CAGR calculations.

    Parameters:
        df (DataFrame): Historical and forecasted financial data.
        assumptions (dict): Explicit growth assumptions for various financial metrics.
        base_year (str): The base year for the forecast (last historical year).
        forecast_years (list): List of years for which the forecast is to be made.
        cagr_period (int): The number of years to use for rolling CAGR calculation.
        
    Returns:
        DataFrame: Forecasted financials including dynamic CAGR for the forecast period.
    """
    
    forecast_df = pd.DataFrame(index=df.index, columns=forecast_years)
    last_values = df[base_year].copy()

    for year_index, year in enumerate(forecast_years):
        # Determine the number of years to look back for CAGR calculation
        start_year = max(int(base_year) - (cagr_period - min(year_index + 1, cagr_period)), int(base_year))
        end_year = int(base_year) + year_index

        for metric in df.index:
            if metric in assumptions and year_index < len(assumptions[metric]['rates']):
                # Use explicit growth rate if available
                growth_rate = assumptions[metric]['rates'][year_index]
            else:
                # Calculate rolling CAGR if not under explicit assumptions or past the available rates
                start_value = df.loc[metric, str(start_year)]
                end_value = df.loc[metric, str(end_year)]
                growth_rate = (end_value / start_value) ** (1 / (end_year - start_year)) - 1 if start_value != 0 else 0
            
            # Calculate forecast value
            base_value = last_values.get(metric, 0)
            forecast_value = base_value * (1 + growth_rate)
            forecast_df.loc[metric, year] = forecast_value
            last_values[metric] = forecast_value

    # Calculate CAGR for the entire forecast period and add as a column
    calculate_cagr(df, forecast_df, base_year, forecast_years)

    return forecast_df

def calculate_cagr(df, forecast_df, base_year, forecast_years):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with historical data.
        forecast_df (DataFrame): The DataFrame with forecast data.
        base_year (str): The starting year for historical data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year, forecast_years[-1]
    num_years = len(forecast_years)
    forecast_df['CAGR'] = ((forecast_df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100

